# Часть 2

In [ ]:
#pip install torch

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from torch import nn
from tqdm import tqdm
import torchvision.transforms as transforms

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic=True

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [4]:
#скачивание датасета для классификации изображений
import torchvision.datasets

CIFAR_train = torchvision.datasets.CIFAR10('./', download = True,
                                           train = True,
                                           transform = transform)
CIFAR_test = torchvision.datasets.CIFAR10('./', download = True,
                                          train = False,
                                          transform = transform)

100%|██████████| 170498071/170498071 [00:01<00:00, 100847105.39it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [5]:
trainloader = torch.utils.data.DataLoader(CIFAR_train, batch_size=20, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(CIFAR_test, batch_size=20, shuffle=False, num_workers=2)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

## Функция для обучения

In [15]:
def fit_model(epochs,
              model,
              trainloader,
              optimizer):
  
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    loss_function = nn.CrossEntropyLoss()
    
    '''данные тренировочной выборки будем перекладывать на cuda по батчам'''
    for epoch in tqdm(range(epochs)):

      for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        X_batch, y_batch = data[0].to(device), data[1].to(device)

        model.train() # перевод в режим train

        optimizer.zero_grad() # обнуляем градиенты

        preds = model.forward(X_batch) # предсказание на батче
        loss_val = loss_function(preds, y_batch) # лосс на батче

        loss_val.backward() # расчет градиентов
        optimizer.step() # шаг градиента + оптимизатора

In [16]:
def get_accuracy(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        return correct / total

## AlexNet

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights

In [ ]:
alexnet_model = alexnet(weights = AlexNet_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:00<00:00, 269MB/s]


In [ ]:
for i in alexnet_model.children():
    for param in i.parameters():
        param.requires_grad = False

In [ ]:
alexnet_model.classifier = nn.Sequential(nn.Dropout(p=0.5, inplace=False),
                                         nn.Linear(in_features=9216, out_features=4096, bias=True),
                                         nn.ReLU(inplace=True),
                                         nn.Dropout(p=0.5, inplace=False),
                                         nn.Linear(in_features=4096, out_features=1024, bias=True),
                                         nn.ReLU(inplace=True),
                                         nn.Linear(in_features=1024, out_features=10, bias=True))

In [ ]:
for i in alexnet_model.children():
    for param in i.parameters():
        print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [ ]:
opt1 = torch.optim.Adam(params = alexnet_model.parameters())

In [ ]:
fit_model(epochs = 10,  
          model = alexnet_model,
          trainloader = trainloader,
          optimizer = opt1)

100%|██████████| 10/10 [16:20<00:00, 98.07s/it]


In [ ]:
alexnet_accuracy = get_accuracy(model = alexnet_model,
                                testloader = testloader)
alexnet_accuracy

0.7874

## GoogleNet

In [ ]:
from torchvision.models import googlenet, GoogLeNet_Weights

In [ ]:
googlenet_model = googlenet(weights = GoogLeNet_Weights.IMAGENET1K_V1)

In [ ]:
for i in googlenet_model.children():
    for param in i.parameters():
        param.requires_grad = False

In [ ]:
googlenet_model.fc = nn.Sequential(nn.Linear(in_features=1024, out_features=512, bias=True),
                                   nn.ReLU(),
                                   nn.Linear(in_features=512, out_features=10, bias=True))

In [ ]:
for i in googlenet_model.children():
    for param in i.parameters():
        if param.requires_grad : print(param.requires_grad)

True
True
True
True


In [ ]:
opt2 = torch.optim.Adam(params = googlenet_model.parameters())

In [ ]:
fit_model(epochs = 10,  
          model = googlenet_model,
          trainloader = trainloader,
          optimizer = opt2)

100%|██████████| 10/10 [18:48<00:00, 112.81s/it]


In [ ]:
googlenet_accuracy = get_accuracy(model = googlenet_model,
                                testloader = testloader)
googlenet_accuracy

0.7366

## VGG 11

In [17]:
from torchvision.models import vgg11, VGG11_Weights

In [18]:
vgg11_model = vgg11(weights = VGG11_Weights.IMAGENET1K_V1)

In [19]:
for i in vgg11_model.children():
    for param in i.parameters():
        param.requires_grad = False

In [20]:
vgg11_model.classifier = nn.Sequential(nn.Linear(in_features=25088, out_features=4096, bias=True),
                                       nn.ReLU(inplace=True),
                                       nn.Dropout(p=0.5, inplace=False),
                                       nn.Linear(in_features=4096, out_features=1024, bias=True),
                                       nn.ReLU(inplace=True),
                                       nn.Dropout(p=0.5, inplace=False),
                                       nn.Linear(in_features=1024, out_features=10, bias=True))

In [21]:
for i in vgg11_model.children():
    for param in i.parameters():
        if param.requires_grad : print(param.requires_grad)

True
True
True
True
True
True


In [22]:
opt3 = torch.optim.Adam(params = vgg11_model.parameters())

In [23]:
fit_model(epochs = 5,  
          model = vgg11_model,
          trainloader = trainloader,
          optimizer = opt3)

100%|██████████| 5/5 [21:05<00:00, 253.09s/it]


In [24]:
vgg11_accuracy = get_accuracy(model = vgg11_model,
                              testloader = testloader)
vgg11_accuracy

0.7937

## ResNet 18

In [31]:
from torchvision.models import resnet18, ResNet18_Weights

In [32]:
resnet18_model = resnet18(weights = ResNet18_Weights.IMAGENET1K_V1)

In [33]:
for i in resnet18_model.children():
    for param in i.parameters():
        param.requires_grad = False

In [34]:
resnet18_model.fc = nn.Linear(in_features=512, out_features=10, bias=True)

In [35]:
opt4 = torch.optim.Adam(params = resnet18_model.parameters())

In [36]:
fit_model(epochs = 10,  
          model = resnet18_model,
          trainloader = trainloader,
          optimizer = opt4)

100%|██████████| 10/10 [16:42<00:00, 100.25s/it]


In [37]:
resnet18_accuracy = get_accuracy(model = resnet18_model,
                                 testloader = testloader)
resnet18_accuracy

0.7511

## Сравнительная таблица

In [38]:
results = pd.DataFrame({'Model': ['AlexNet', 'GoogleNet', 'VGG 11', 'ResNet 18'],
                        'Number of params': ['61.1M', '6.6M', '132.9M', '11.7M'],
                        'Epochs': [10, 10, 5, 10],
                        'Train time': ['16:02', '18:48', '21.05', '16:42'],
                        'Accuracy': [0.787, 0.736, 0.793, 0.751]})

In [39]:
results

,Model,Number of params,Epochs,Train time,Accuracy
0,AlexNet,61.1M,10,16:02,0.787
1,GoogleNet,6.6M,10,18:48,0.736
2,VGG 11,132.9M,5,21.05,0.793
3,ResNet 18,11.7M,10,16:42,0.751
